In [ ]:
%pylab inline

In [1]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan, DBSCAN
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import multiprocessing
from joblib import Parallel, delayed
import seaborn as sns
import pandas as pd

import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.collections import PatchCollection
import matplotlib.gridspec as gridspec


from astropy import units as u
from astropy.coordinates import SkyCoord

import warnings
warnings.filterwarnings('ignore')

import loaders

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import scipy.spatial as sp

# Voro groups

In [ ]:
groups_voro, galxs_voro = loaders.load_voro('Catalogos/Voro/clusparam_v.dat','Catalogos/Voro/miensalclumag_v.dat')

In [ ]:
groups_voro.head(10)

## VVVIRGC

In [ ]:
df = pd.read_csv("Catalogos/final_cat_v4_October_myver.txt",delim_whitespace=True)

In [ ]:
c = SkyCoord(ra=df['ALPHA_J2000'], dec=df['DELTA_J2000'], frame='icrs',unit='degree')
df['l'] = c.galactic.l
df['b'] = c.galactic.b

In [ ]:
area = pd.read_csv("Catalogos/Voro/areas.dat",delim_whitespace=True)
df['area'] = area['Area']

## Coordenadas galácticas

In [ ]:
fig = plt.figure(figsize=(30,10))
ax = fig.add_subplot(111,aspect='equal')
ax.plot(df['l'],df['b'],'.',alpha=1.0,markersize=3)
ax.set_xlim(df['l'].min(),df['l'].max())
ax.set_ylim(df['b'].min(),df['b'].max())
ax.set_xlabel('l [º]',fontsize=16)
ax.set_ylabel('b [º]',fontsize=16);
ax.tick_params(labelsize=16, width=3)

In [ ]:
X = df[['l','b']]

In [ ]:
voro = sp.Voronoi(X)

In [ ]:
def plot_grupos(ax):
    unique_labels = groups_voro['nmem'].values
    norm = mcolors.LogNorm(4,91)
    colors = [plt.cm.jet(each) for each in norm(unique_labels)]
    
    for k, row in groups_voro.iterrows():
        radius = row['radius']
        circle = mpatches.Circle((row['l'],row['b']),lw=2,radius=radius/60,
                             edgecolor=plt.cm.jet(norm(row['nmem'])),facecolor='None')
        ax.add_patch(circle)
    return(norm)

In [ ]:
def plot_polygons(voro,gals,ax=None):
    minima = 0.60
    maxima = 43.45
    
    norm = mpl.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap='Blues_r')
    
    _area = np.zeros(len(voro.point_region)) - 1.0
    for r in range(voro.npoints):
        region = voro.regions[voro.point_region[r]]
        if (not -1 in region) and (gals['area'].iloc[r] > 0.0):
            polygon = [voro.vertices[i] for i in region]
            a = np.array(polygon)
            xc = a[:,0].sum()/len(a)
            yc = a[:,1].sum()/len(a)
            xs = a[:,0] - xc
            ys = a[:,1] - yc
            theta = np.arctan2(ys,xs)*180/np.pi
            indx = np.argsort(theta)
            xs = a[indx,0]
            ys = a[indx,1]
            i=np.arange(len(xs))
            Area=np.abs(np.sum(xs[i-1]*ys[i]-xs[i]*ys[i-1])*0.5)
            _area[r] = Area
            ax.fill(*zip(*polygon), color=mapper.to_rgba(gals['area'].iloc[r]))

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(211,aspect='equal')

plot_polygons(voro,df,ax=ax)
plot_grupos(ax)

p = sp.voronoi_plot_2d(voro,ax=ax,show_points=False,show_vertices=False,line_alpha=0.1)

ax.set_xlim(df['l'].max(),322.0)
ax.set_ylim(df['b'].min(),df['b'].max())
ax.set_xlabel('Galactic Longitude [deg]',fontsize=16)
ax.set_ylabel('Galactic Latitude [deg]',fontsize=16);
ax.tick_params(labelsize=16, width=3)
ax.grid(True)

ax = fig.add_subplot(212,aspect='equal')

plot_polygons(voro,df,ax=ax)
p = sp.voronoi_plot_2d(voro,ax=ax,show_points=False,show_vertices=False,line_alpha=0.1)
norm = plot_grupos(ax)


ax.set_xlim(322.0,df['l'].min())
ax.set_ylim(df['b'].min(),df['b'].max())
ax.set_xlabel('Galactic Longitude [deg]',fontsize=16)
ax.set_ylabel('Galactic Latitude [deg]',fontsize=16);
ax.tick_params(labelsize=16, width=3)
ax.grid(True)

#plt.subplots_adjust(bottom=0.1,right=0.8,top=0.9)
cax = plt.axes([0.93, 0.15, 0.01, 0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=plt.cm.jet),cax=cax,ticks=[4,10,30,91])
cbar.ax.set_yticklabels(['4','10','30','91'])
cbar.set_label('Number of members')